<a href="https://colab.research.google.com/github/sajidcsecu/radioGenomic/blob/main/PreprocessedForSegmentation(Clipping_and_Uniform_Spacing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary packages

In [4]:
!pip install SimpleITK

In [5]:
import os
import h5py
import torch
import numpy as np
import SimpleITK as sitk
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import multiprocessing as mp
from google.colab import drive
import matplotlib.pyplot as plt
from pathlib import Path
from skimage import measure
from typing import Optional
import os

In [6]:
class HDF5Preprocessor:
    def __init__(self, input_path, output_path, spacing=(1.0, 1.0, 1.0), clip_range=(-1350, 150), device='cuda', max_patients=None):
        self.input_path = input_path
        self.output_path = output_path
        self.spacing = spacing
        self.clip_range = clip_range
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.max_patients = max_patients  # for debugging or small runs

    def run(self):
        os.makedirs(os.path.dirname(self.output_path), exist_ok=True)

        with h5py.File(self.input_path, 'r') as in_file, h5py.File(self.output_path, 'w') as out_file:
            patient_ids = list(in_file.keys())
            print(f"🔍 Found {len(patient_ids)} patients in {os.path.basename(self.input_path)}.")

            if not patient_ids:
                raise ValueError("❌ No patient groups found in input HDF5 file.")

            for i, pid in enumerate(tqdm(patient_ids, desc=f"Processing {os.path.basename(self.input_path)}")):
                if self.max_patients is not None and i >= self.max_patients:
                    break
                try:
                    in_group = in_file[pid]
                    out_group = out_file.create_group(pid)
                    self.process_patient(in_group, out_group)
                except Exception as e:
                    print(f"❌ [ERROR] Failed to process {pid}: {e}")

    def process_patient(self, in_group, out_group):
        ct_np = in_group['ct'][:]
        mask_np = in_group['mask'][:]
        print(f"📄 {in_group.name} — CT shape: {ct_np.shape}, Mask shape: {mask_np.shape}")

        # Validate required metadata
        required_attrs = [
            'ct_origin', 'ct_spacing', 'ct_direction',
            'mask_origin', 'mask_spacing', 'mask_direction', 'shape'
        ]
        for attr in required_attrs:
            if attr not in in_group.attrs:
                raise KeyError(f"Missing required attribute: {attr}")

        # Convert to SimpleITK images
        sitk_ct = sitk.GetImageFromArray(ct_np)
        sitk_mask = sitk.GetImageFromArray(mask_np.astype(np.uint8))

        sitk_ct.SetOrigin(in_group.attrs['ct_origin'])
        sitk_ct.SetSpacing(in_group.attrs['ct_spacing'])
        sitk_ct.SetDirection(in_group.attrs['ct_direction'])

        sitk_mask.SetOrigin(in_group.attrs['mask_origin'])
        sitk_mask.SetSpacing(in_group.attrs['mask_spacing'])
        sitk_mask.SetDirection(in_group.attrs['mask_direction'])

        # Resample
        resampled_ct, resampled_mask = self.resample_image_and_mask(sitk_ct, sitk_mask)

        # Convert back to NumPy arrays
        res_ct_np = sitk.GetArrayFromImage(resampled_ct)
        res_mask_np = sitk.GetArrayFromImage(resampled_mask)

        # Clip CT intensities
        res_ct_np = np.clip(res_ct_np, *self.clip_range).astype(np.float32)

        # Compute and store min/max of clipped CT
        ct_min = float(res_ct_np.min())
        ct_max = float(res_ct_np.max())

        # Save to HDF5
        out_group.create_dataset('ct', data=res_ct_np, compression='gzip', dtype=np.float32)
        out_group.create_dataset('mask', data=res_mask_np, compression='gzip', dtype=np.uint8)

        # Metadata
        out_group.attrs['shape'] = res_ct_np.shape
        out_group.attrs['ct_origin'] = resampled_ct.GetOrigin()
        out_group.attrs['ct_spacing'] = resampled_ct.GetSpacing()
        out_group.attrs['ct_direction'] = resampled_ct.GetDirection()
        out_group.attrs['mask_origin'] = resampled_mask.GetOrigin()
        out_group.attrs['mask_spacing'] = resampled_mask.GetSpacing()
        out_group.attrs['mask_direction'] = resampled_mask.GetDirection()
        out_group.attrs['ct_min'] = ct_min
        out_group.attrs['ct_max'] = ct_max

        print(f"✅ Saved {in_group.name} with shape {res_ct_np.shape}, min: {ct_min:.1f}, max: {ct_max:.1f}")

    def resample_image_and_mask(self, image, mask, output_size_xy=(512, 512)):
        original_spacing = image.GetSpacing()
        original_size = image.GetSize()

        spacing_x, spacing_y, spacing_z = self.spacing
        out_size_x, out_size_y = output_size_xy

        # Compute output depth (z-dimension)
        physical_depth = original_size[2] * original_spacing[2]
        out_size_z = int(round(physical_depth / spacing_z))
        new_size = [out_size_x, out_size_y, out_size_z]

        # Configure resampler
        resampler = sitk.ResampleImageFilter()
        resampler.SetOutputSpacing(self.spacing)
        resampler.SetSize(new_size)
        resampler.SetOutputOrigin(image.GetOrigin())
        resampler.SetOutputDirection(image.GetDirection())

        # Interpolate CT with B-Spline, mask with NearestNeighbor
        resampler.SetInterpolator(sitk.sitkBSpline)
        resampled_image = resampler.Execute(image)

        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        resampled_mask = resampler.Execute(mask)

        return resampled_image, resampled_mask


2. Mount Drive

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


3. Main Block

In [8]:
if __name__ == "__main__":
    base_input_dir = "/content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_With_Empty_NonEmpty_slices_In_Train"
    base_output_dir = "/content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train"

    splits = ['train_dataset.hdf5', 'valid_dataset.hdf5', 'test_dataset.hdf5']

    for split in splits:
        input_file = os.path.join(base_input_dir, split)
        output_file = os.path.join(base_output_dir, split)

        print(f"\n🚀 Starting preprocessing: {split}")
        preprocessor = HDF5Preprocessor(input_path=input_file, output_path=output_file)
        preprocessor.run()
        print(f"📁 Finished saving preprocessed data to: {output_file}")


🚀 Starting preprocessing: train_dataset.hdf5
🔍 Found 340 patients in train_dataset.hdf5.


Processing train_dataset.hdf5:   0%|          | 0/340 [00:00<?, ?it/s]

📄 /LUNG1-002 — CT shape: (111, 512, 512), Mask shape: (111, 512, 512)


Processing train_dataset.hdf5:   0%|          | 1/340 [00:32<3:02:04, 32.23s/it]

✅ Saved /LUNG1-002 with shape (333, 512, 512), min: -1350.0, max: 150.0
📄 /LUNG1-003 — CT shape: (107, 512, 512), Mask shape: (107, 512, 512)


...

✅ Saved /LUNG1-405 with shape (402, 512, 512), min: -1270.6, max: 150.0
📄 /LUNG1-408 — CT shape: (107, 512, 512), Mask shape: (107, 512, 512)


Processing test_dataset.hdf5:  97%|█████████▋| 37/38 [20:14<00:33, 33.97s/it]

✅ Saved /LUNG1-408 with shape (321, 512, 512), min: -1350.0, max: 150.0
📄 /LUNG1-410 — CT shape: (134, 512, 512), Mask shape: (134, 512, 512)


Processing test_dataset.hdf5: 100%|██████████| 38/38 [20:51<00:00, 32.92s/it]

✅ Saved /LUNG1-410 with shape (402, 512, 512), min: -1282.3, max: 150.0
📁 Finished saving preprocessed data to: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train/test_dataset.hdf5


In [9]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage import measure
from typing import Optional


class PreprocessedSliceViewer:
    def __init__(self, original_hdf5_path: str, preprocessed_hdf5_path: str):
        self.original_path = Path(original_hdf5_path)
        self.preprocessed_path = Path(preprocessed_hdf5_path)
        self.original_data = self._load_hdf5(self.original_path)
        self.preprocessed_data = self._load_hdf5(self.preprocessed_path)

    def _load_hdf5(self, path: Path):
        if not path.exists():
            raise FileNotFoundError(f"HDF5 file not found: {path}")
        return h5py.File(path, 'r')

    def _get_contour(self, mask_slice: np.ndarray):
        return measure.find_contours(mask_slice, level=0.5)

    def show_patient_slices_comparison(
        self,
        patient_id: str,
        save_path: Optional[str] = None,
        show_mask_overlay: bool = True,
    ):
        L = -600
        W = 1500
        vmin = L - W // 2
        vmax = L + W // 2

        if patient_id not in self.original_data or patient_id not in self.preprocessed_data:
            raise KeyError(f"Patient ID {patient_id} not found in both HDF5 files.")

        orig_ct = self.original_data[patient_id]["ct"][()]
        orig_mask = self.original_data[patient_id]["mask"][()]
        prepro_ct = self.preprocessed_data[patient_id]["ct"][()]
        prepro_mask = self.preprocessed_data[patient_id]["mask"][()]

        # Get all non-empty slices
        non_empty_indices_org = [i for i in range(orig_mask.shape[0]) if np.sum(orig_mask[i]) > 0]
        non_empty_indices_prepro = [i for i in range(prepro_mask.shape[0]) if np.sum(prepro_mask[i]) > 0]
        if len(non_empty_indices_org) == 0:
            raise ValueError(f"No non-empty mask slices found for patient {patient_id}.")

        num_slices_org = len(non_empty_indices_org)
        print("Number of Non-Empty Slices of original: ", num_slices_org)
        print("Number of Non-Empty Slices of preprocessed: ", len(non_empty_indices_prepro))
        print("Spacing of Original: ", self.original_data[patient_id].attrs['ct_spacing'])
        print("Spacing of Preprocessed: ", self.preprocessed_data[patient_id].attrs['ct_spacing'])
        print("Origin of Original: ", self.original_data[patient_id].attrs['ct_origin'])
        print("Origin of Preprocessed: ", self.preprocessed_data[patient_id].attrs['ct_origin'])
        print("Direction of Original: ", self.original_data[patient_id].attrs['ct_direction'])
        print("Direction of Preprocessed: ", self.preprocessed_data[patient_id].attrs['ct_direction'])
        print("Shape of Original: ", self.original_data[patient_id].attrs['shape'])
        print("Shape of Preprocessed: ", self.preprocessed_data[patient_id].attrs['shape'])

        # Create subplots
        fig, axes = plt.subplots(num_slices_org, 2, figsize=(10, 3 * num_slices_org))

        # Ensure axes is always 2D
        if num_slices_org == 1:
            axes = np.expand_dims(axes, axis=0)

        for i, slice_idx in enumerate(non_empty_indices_org):
            for j, (ct, mask, label) in enumerate([
                (orig_ct, orig_mask, "Original"),
                (prepro_ct, prepro_mask, "Preprocessed")
            ]):
                ax = axes[i, j]
                ax.imshow(ct[slice_idx], cmap="gray", vmin=vmin, vmax=vmax)

                if show_mask_overlay:
                    for contour in self._get_contour(mask[slice_idx]):
                        ax.plot(contour[:, 1], contour[:, 0], linewidth=1.5, color="red")
                ax.text(
                  10, 30, f"Slice {slice_idx}",
                  color="yellow",
                  fontsize=12,
                  bbox=dict(facecolor='black', alpha=0.5)
              )
                if i == 0:
                    ax.set_title(label, fontsize=12)

                ax.axis("off")

        plt.tight_layout()
        if save_path:
            Path(save_path).parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_path, format="pdf")
            print(f"[INFO] Saved plot to: {save_path}")
        else:
            plt.show()

        plt.close()

    def show_patient_slices(
        self,
        patient_id: str,
        save_path: Optional[str] = None,
        original: bool = True,
    ):

        # Windowing
        L = -600
        W = 1500
        vmin = L - W // 2
        vmax = L + W // 2

        # Load data
        if patient_id not in self.original_data or patient_id not in self.preprocessed_data:
            raise KeyError(f"Patient ID {patient_id} not found in both HDF5 files.")

        if original:
            ct = self.original_data[patient_id]["ct"][()]
            mask = self.original_data[patient_id]["mask"][()]
            label = "Original"
        else:
            ct = self.preprocessed_data[patient_id]["ct"][()]
            mask = self.preprocessed_data[patient_id]["mask"][()]
            label = "Preprocessed"

        # Get all non-empty slices
        non_empty_indices = [i for i in range(mask.shape[0]) if np.sum(mask[i]) > 0]
        if not non_empty_indices:
            raise ValueError(f"No non-empty mask slices found for patient {patient_id}.")

        num_slices = len(non_empty_indices)
        print("Number of Non-Empty Slices: ", num_slices)

        # Create subplot grid: rows = slices, cols = [CT, mask, CT+overlay]
        fig, axes = plt.subplots(num_slices, 3, figsize=(15, 3 * num_slices))
        if num_slices == 1:
            axes = np.expand_dims(axes, axis=0)  # Make 2D iterable for consistency

        for row, slice_idx in enumerate(non_empty_indices):
            # --- Column 1: CT ---
            ax_ct = axes[row][0]
            ax_ct.imshow(ct[slice_idx], cmap="gray", vmin=vmin, vmax=vmax)
            ax_ct.set_title("CT", fontsize=10)
            ax_ct.text(10, 30, f"Slice {slice_idx}", color="yellow", fontsize=10,
                      bbox=dict(facecolor='black', alpha=0.5))
            ax_ct.axis("off")

            # --- Column 2: Mask ---
            ax_mask = axes[row][1]
            ax_mask.imshow(mask[slice_idx], cmap="gray")
            ax_mask.set_title("Mask", fontsize=10)
            ax_mask.axis("off")

            # --- Column 3: CT with Overlay ---
            ax_overlay = axes[row][2]
            ax_overlay.imshow(ct[slice_idx], cmap="gray", vmin=vmin, vmax=vmax)
            for contour in self._get_contour(mask[slice_idx]):
                ax_overlay.plot(contour[:, 1], contour[:, 0], linewidth=1.5, color="red")
            ax_overlay.set_title("Overlay", fontsize=10)
            ax_overlay.axis("off")

        # Add global title
        fig.suptitle(f"{label} slices for Patient {patient_id}", fontsize=14)
        plt.tight_layout(rect=[0, 0, 1, 0.98])  # leave space for suptitle

        # Save or show
        if save_path:
            Path(save_path).parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_path, format="pdf")
            print(f"[INFO] Saved plot to: {save_path}")
        else:
            plt.show()

        plt.close()


In [10]:
if __name__ == "__main__":
    original_hdf5 = "/content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_With_Empty_NonEmpty_slices_In_Train/train_dataset.hdf5"
    preprocessed_hdf5 = "/content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train/train_dataset.hdf5"
    patient_id = "LUNG1-009"

    viewer = PreprocessedSliceViewer(original_hdf5_path=original_hdf5 ,
                                     preprocessed_hdf5_path=preprocessed_hdf5)

    viewer.show_patient_slices_comparison(
        patient_id=patient_id,
        save_path=f"/content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train/{patient_id}_all_non_empty_slices1.pdf"
    )

    viewer.show_patient_slices(
        patient_id=patient_id,
        save_path=f"/content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train/Comparison_Plots/original_{patient_id}_all_non_empty_slices1.pdf",
        original=True
    )

    viewer.show_patient_slices(
        patient_id=patient_id,
        save_path=f"/content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train/Comparison_Plots/Preprocessed_{patient_id}_all_non_empty_slices1.pdf",
        original=False
    )


Number of Non-Empty Slices of original:  22
Number of Non-Empty Slices of preprocessed:  66
Spacing of Original:  [0.977 0.977 3.   ]
Spacing of Preprocessed:  [1. 1. 1.]
Origin of Original:  [-250.112 -250.112 -161.9  ]
Origin of Preprocessed:  [-250.112 -250.112 -161.9  ]
Direction of Original:  [1. 0. 0. 0. 1. 0. 0. 0. 1.]
Direction of Preprocessed:  [1. 0. 0. 0. 1. 0. 0. 0. 1.]
Shape of Original:  [105 512 512]
Shape of Preprocessed:  [315 512 512]
[INFO] Saved plot to: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train/LUNG1-009_all_non_empty_slices1.pdf
Number of Non-Empty Slices:  22
[INFO] Saved plot to: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train/Comparison_Plots/original_LUNG1-009_all_non_empty_slices1.pdf
Number of Non-Empty Slices:  66
[INFO] Saved plot to: /content/drive/MyDrive/PhDwork/Segmentati

In [11]:
import h5py
import os
from typing import Dict, Tuple

class SliceCounter:
    def __init__(self, hdf5_path: str):
        self.hdf5_path = hdf5_path

    def count_slices(self) -> Tuple[int, int]:
        """Returns (non_empty_slices, empty_slices)"""
        non_empty = 0
        empty = 0
        with h5py.File(self.hdf5_path, 'r') as f:
            for pid in f.keys():
                masks = f[pid]['mask'][:]  # Shape: (slices, H, W)
                sums = masks.sum(axis=(1, 2))  # Sum over H and W per slice
                non_empty += (sums > 0).sum()
                empty += (sums == 0).sum()
        return non_empty, empty


class DatasetAnalyzer:
    def __init__(self, dataset_dir: str):
        self.dataset_dir = dataset_dir
        self.splits = {
            "train": os.path.join(dataset_dir, "train_dataset.hdf5"),
            "valid": os.path.join(dataset_dir, "valid_dataset.hdf5"),
            "test": os.path.join(dataset_dir, "test_dataset.hdf5")
        }

    def analyze(self) -> Dict[str, Dict[str, int]]:
        results = {}
        for split_name, path in self.splits.items():
            if not os.path.exists(path):
                print(f"⚠️ Skipping missing file: {path}")
                continue
            print(f"🔍 Counting slices in: {split_name}")
            counter = SliceCounter(path)
            non_empty, empty = counter.count_slices()
            results[split_name] = {
                "non_empty": non_empty,
                "empty": empty,
                "total": non_empty + empty
            }
        return results


if __name__ == "__main__":
    dataset_dir = "/content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train"

    analyzer = DatasetAnalyzer(dataset_dir)
    slice_counts = analyzer.analyze()

    print("\n✅ Slice Count Summary:")
    for split, stats in slice_counts.items():
        print(f"\n📁 {split.capitalize()} Set")
        print(f"  Total Slices   : {stats['total']}")
        print(f"  Non-Empty Slices: {stats['non_empty']}")
        print(f"  Empty Slices   : {stats['empty']}")


🔍 Counting slices in: train
🔍 Counting slices in: valid
🔍 Counting slices in: test

✅ Slice Count Summary:

📁 Train Set
  Total Slices   : 124084
  Non-Empty Slices: 17574
  Empty Slices   : 106510

📁 Valid Set
  Total Slices   : 15423
  Non-Empty Slices: 2616
  Empty Slices   : 12807

📁 Test Set
  Total Slices   : 13956
  Non-Empty Slices: 1848
  Empty Slices   : 12108
